In [1]:
#flu selections

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', None)

import os
import matplotlib
import seaborn as sns

from Bio import SeqIO
from Bio.Seq import Seq

import warnings
import itertools
import numpy as np

import tqdm
import gc

import scipy as sp

In [2]:
warnings.simplefilter('ignore')

In [10]:
#load in the sample information table
samples=pd.read_csv('data/samples.csv')
samples

,sample_name,virus_background,replicate,selection,sample_id,Lane,inline_fwd,inline_rev,read_1_file,read_2_file
0,USSR77_R1_NoAb_L1,USSR77,R1,NoAb,1,1,TTGATCG,TACCTGA,data/sequencing_data/NoAb_USSR77_1_S1_L001_R1_...,data/sequencing_data/NoAb_USSR77_1_S1_L001_R2_...
1,USSR77_R1_NoAb_L2,USSR77,R1,NoAb,1,2,TTGATCG,TACCTGA,data/sequencing_data/NoAb_USSR77_1_S1_L002_R1_...,data/sequencing_data/NoAb_USSR77_1_S1_L002_R2_...
2,USSR77_R2_NoAb_L1,USSR77,R2,NoAb,2,1,CATTCTCGAG,TACCTGA,data/sequencing_data/NoAb_USSR77_2_S2_L001_R1_...,data/sequencing_data/NoAb_USSR77_2_S2_L001_R2_...
3,USSR77_R2_NoAb_L2,USSR77,R2,NoAb,2,2,CATTCTCGAG,TACCTGA,data/sequencing_data/NoAb_USSR77_2_S2_L002_R1_...,data/sequencing_data/NoAb_USSR77_2_S2_L002_R2_...
4,SN89_R1_NoAb_L1,SN89,R1,NoAb,3,1,CCTTAGTACTA,TACCTGA,data/sequencing_data/NoAb_SN89_1_S3_L001_R1_00...,data/sequencing_data/NoAb_SN89_1_S3_L001_R2_00...
...,...,...,...,...,...,...,...,...,...,...
123,SYD21_R2_ADI-77470_L2,SYD21,R2,ADI-77470,62,2,GATGTCCT,GAGATCAC,data/sequencing_data/ADI-77470_SYD21_2_S62_L00...,data/sequencing_data/ADI-77470_SYD21_2_S62_L00...
124,SYD21_R1_ADI-86325_L1,SYD21,R1,ADI-86325,63,1,CTCATCCAG,GAGATCAC,data/sequencing_data/ADI-86325_SYD21_1_S63_L00...,data/sequencing_data/ADI-86325_SYD21_1_S63_L00...
125,SYD21_R1_ADI-86325_L2,SYD21,R1,ADI-86325,63,2,CTCATCCAG,GAGATCAC,data/sequencing_data/ADI-86325_SYD21_1_S63_L00...,data/sequencing_data/ADI-86325_SYD21_1_S63_L00...
126,SYD21_R2_ADI-86325_L1,SYD21,R2,ADI-86325,64,1,GACGGAACTC,GAGATCAC,data/sequencing_data/ADI-86325_SYD21_2_S64_L00...,data/sequencing_data/ADI-86325_SYD21_2_S64_L00...


In [6]:
#run PEAR for paired end alignments for all samples
for idx,row in samples.iterrows():
    sample_name = row['sample_name']
    fwd_read = row['read_1_file']
    rev_read = row['read_2_file']
    os.system(f'pear -f {fwd_read} -r {rev_read} -o "data/sequencing_data/pear_output/"{sample_name} -j 6')

 ____  _____    _    ____ 
|  _ \| ____|  / \  |  _ \
| |_) |  _|   / _ \ | |_) |
|  __/| |___ / ___ \|  _ <
|_|   |_____/_/   \_\_| \_\

PEAR v0.9.6 [January 15, 2015]

Citation - PEAR: a fast and accurate Illumina Paired-End reAd mergeR
Zhang et al (2014) Bioinformatics 30(5): 614-620 | doi:10.1093/bioinformatics/btt593

Forward reads file.................: data/sequencing_data/CH65_ND_SN89_1_S15_L001_R1_001.fastq.gz
Reverse reads file.................: data/sequencing_data/CH65_ND_SN89_1_S15_L001_R2_001.fastq.gz
PHRED..............................: 33
Using empirical frequencies........: YES
Statistical method.................: OES
Maximum assembly length............: 999999
Minimum assembly length............: 50
p-value............................: 0.010000
Quality score threshold (trimming).: 0
Minimum read size after trimming...: 1
Maximal ratio of uncalled bases....: 1.000000
Minimum overlap....................: 10
Scoring method.....................: Scaled score
Threads......

In [15]:
# for each sample, extract barcode and output with the number of counts based on umis
flanks=['AGCCTCCAGTGTAGGATTTGCATTTAA','TGGTGCTTTTGGTCTCCCTGGGG']

for index,row in samples.iterrows():
    gc.collect()
    sample_name = row['sample_name']
    virus_background = row['virus_background']
    replicate = row['replicate']
    selection = row['selection']
    inline_fwd = row['inline_fwd']
    inline_rev = row['inline_rev']
    inline_rev_rcomp = str(Seq(inline_rev).reverse_complement())
    
    print(f'Now processing: {sample_name}')
    
    required_length = 78 + 16 + len(inline_fwd) + len(inline_rev)
    barcode_umi_counts = dict()
    for record in tqdm.tqdm(SeqIO.parse(f'data/sequencing_data/pear_output/{sample_name}.assembled.fastq', 'fastq')):
        seq_str = str(record.seq)
        if len(seq_str) == required_length and 'N' not in seq_str and inline_fwd in seq_str[8:8+len(inline_fwd)] and inline_rev_rcomp in seq_str[-8-len(inline_rev_rcomp):-8]:
            UMI_merged = seq_str[:8]+seq_str[-8:]
            if flanks[0] in seq_str and flanks[1] in seq_str:
                barcode = seq_str.split(flanks[0])[1].split(flanks[1])[0]                   
            elif flanks[0] in seq_str and flanks[1] not in seq_str:
                barcode = seq_str.split(flanks[0])[1][:28]
            elif flanks[0] not in seq_str and flanks[1] in seq_str:
                barcode = seq_str.split(flanks[1])[0][-28:]
                
            barcode_umi_counts.setdefault(barcode, dict())
            barcode_umi_counts[barcode].setdefault(UMI_merged, 0)
            barcode_umi_counts[barcode][UMI_merged] += 1
            
    data = {'barcode':[], 'umi':[], 'count':[]}
    for barcode, umis in barcode_umi_counts.items():
        for umi, count in umis.items():
            data['barcode'].append(barcode)
            data['umi'].append(umi)
            data['count'].append(count)
    data = pd.DataFrame(data)
    #data['sample_name'] = row['sample_name']
    data['sample'] = row['virus_background']+'_'+row['replicate']+'_'+row['selection']
    data['virus_background'] = row['virus_background']
    data['replicate'] = row['replicate']
    data['selection'] = row['selection']
    
    print("Dataframe length:",len(data))
    
    data.to_csv(f'data/barcode_umi_counts/{sample_name}_barcodes_umis.csv')

Now processing: USSR77_R1_NoAb_L1


6829711it [01:12, 94400.97it/s]


Dataframe length: 5716580
Now processing: USSR77_R1_NoAb_L2


6865796it [01:14, 92479.21it/s]


Dataframe length: 5772280
Now processing: USSR77_R2_NoAb_L1


...

Dataframe length: 6416084
Now processing: SYD21_R2_ADI-86325_L1


4647522it [00:49, 93800.52it/s]


Dataframe length: 3906486
Now processing: SYD21_R2_ADI-86325_L2


4664855it [00:49, 93551.79it/s]


Dataframe length: 3941819


In [16]:
#generate sample list
sample_list=[]
for idx,row in samples.iterrows():
    sample=row['sample_name'][:-3]
    if sample in sample_list:
        continue
    if sample not in sample_list:
        sample_list.append(sample)

In [17]:
#for each sample, merge data from both lanes of novaseq sp, and dropping duplicates of the same barcode_umi combination
#uncomment elif statements if data with only single lanes is being used

all_data_df=pd.DataFrame()

for f in tqdm.tqdm(sample_list):
    gc.collect()
    file_lane1 = f+'_L1_barcodes_umis.csv'
    file_lane2 = f+'_L2_barcodes_umis.csv'
    
    if file_lane1 and file_lane2 in os.listdir('data/barcode_umi_counts'):  
        temp_df1 = pd.read_csv(f'data/barcode_umi_counts/{file_lane1}')
        temp_df2 = pd.read_csv(f'data/barcode_umi_counts/{file_lane2}')
        temp_df = pd.concat([temp_df1,temp_df2])
        temp_df['barcode_umi']=temp_df['barcode']+'_'+temp_df['umi']
        temp_df = temp_df.drop_duplicates(subset=['barcode_umi','sample'],keep='first')
        temp_df['bc_count']=1
        temp_df_agg = temp_df.groupby(['virus_background','replicate','selection','sample','barcode']).agg({'bc_count':'sum'}).reset_index()
        all_data_df=pd.concat([all_data_df,temp_df_agg]).reset_index(drop=True)
    
    #elif file_lane1 in os.listdir('data/barcode_umi_counts') and file_lane2 not in os.listdir('data/barcode_umi_counts'):  
        #temp_df = pd.read_csv(f'data/barcode_umi_counts/{file_lane1}')
        #temp_df['bc_count']=1
        #temp_df['barcode_umi']=temp_df['barcode']+'_'+temp_df['umi']
        #temp_df_agg = temp_df.groupby(['virus_background','replicate','selection','sample','barcode']).agg({'bc_count':'sum'}).reset_index()
        #all_data_df=pd.concat([all_data_df,temp_df_agg]).reset_index(drop=True)
        
    #elif file_lane2 in os.listdir('data/barcode_umi_counts') and file_lane1 not in os.listdir('data/barcode_umi_counts'):  
        #temp_df = pd.read_csv(f'data/barcode_umi_counts/{file_lane2}')
        #temp_df['bc_count']=1
        #temp_df['barcode_umi']=temp_df['barcode']+'_'+temp_df['umi']
        #temp_df_agg = temp_df.groupby(['virus_background','replicate','selection','sample','barcode']).agg({'bc_count':'sum'}).reset_index()
        #all_data_df=pd.concat([all_data_df,temp_df_agg]).reset_index(drop=True)
    
    else:
        continue

all_data_df.to_csv('data/all_data_df.csv',index=False)
all_data_df

100%|██████████| 64/64 [16:55<00:00, 15.87s/it]


,virus_background,replicate,selection,sample,barcode,bc_count
0,USSR77,R1,NoAb,USSR77_R1_NoAb,AAAAAAAGCTCGAGCTTGGCCGTACAGT,3
1,USSR77,R1,NoAb,USSR77_R1_NoAb,AAAAACGGCGATAGCTGAAAAATTCACG,1
2,USSR77,R1,NoAb,USSR77_R1_NoAb,AAAACATGCCATAGCTGAAAACGATACT,1
3,USSR77,R1,NoAb,USSR77_R1_NoAb,ACAAAAAGCTCGAGCTACTGCAGGCGCT,2
4,USSR77,R1,NoAb,USSR77_R1_NoAb,ACAAAAAGCTCGAGCTATTAACAAGTCC,1
...,...,...,...,...,...,...
2755098,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGACATAGCTCCTGATCAGGTC,1
2755099,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGCTCAAGCTCGAAGTTTAGAG,1
2755100,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGCTCAAGCTGCGGTTACCGGC,3
2755101,SYD21,R2,ADI-86325,SYD21_R2_ADI-86325,TTGTGTTGCTCAAGCTGTATGTCGGCTT,1
